# Quantization 비교 실험 - EchoNet-Dynamic EF 회귀 모델

이 노트북은 Post-Training Quantization (PTQ)와 Quantization-Aware Training (QAT)를 비교하는 실험을 수행합니다.

## 실험 목표
- Baseline (FP32) 모델 학습
- PTQ 적용 및 성능 측정
- QAT 적용 및 성능 측정
- 세 모델의 MAE, Model Size, Latency 비교



## 1. 환경 체크 및 설정


In [ ]:
# GPU/CPU 체크
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("Using CPU")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\nUsing device: {device}")


In [ ]:
# 필요한 패키지 설치
%pip install -q opencv-python pandas tqdm matplotlib seaborn


In [ ]:
# 방법 1: GitHub에서 클론 (권장)
!git clone https://github.com/5seoyoung/2025_edge_computing_task2.git
import sys
sys.path.append('/content/2025_edge_computing_task2')

# 방법 2: 파일 직접 업로드
# from google.colab import files
# files.upload()  # config.py, dataset.py, model.py 등을 업로드
# sys.path.append('/content')

# 현재 디렉토리 확인
import os
print(f"Current directory: {os.getcwd()}")
print(f"Files in current directory: {os.listdir('.')}")


## 2. 데이터 준비

### 옵션 A: Google Drive 마운트 (권장)


In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 전체 데이터 경로 설정 (Drive에 전체 데이터가 있는 경우)
# 샘플 데이터를 사용하려면 아래 "옵션 B: 샘플 데이터 사용" 셀을 사용하세요
FULL_DATA_ROOT = "/content/drive/MyDrive/echonet_dynamic"  # 여기를 실제 경로로 수정하세요

# 전체 데이터를 사용할 경우
# DATA_ROOT = FULL_DATA_ROOT


### 옵션 B: 샘플 데이터 사용 (전체 데이터가 너무 큰 경우)

이미 Drive에 준비된 샘플 데이터를 사용하거나, 아래 코드로 샘플 데이터를 생성할 수 있습니다.


In [ ]:
# 샘플 데이터 사용 설정
USE_SAMPLE_DATA = True  # True: 샘플 데이터 사용, False: 전체 데이터 사용

if USE_SAMPLE_DATA:
    # 이미 Drive에 준비된 샘플 데이터 경로
    # 폴더 구조: sample_echonet_dynamic/Videos/*.avi, sample_echonet_dynamic/FileList.csv
    DATA_ROOT = "/content/drive/MyDrive/sample_echonet_dynamic"  # 여기를 실제 경로로 수정
    
    from pathlib import Path
    sample_path = Path(DATA_ROOT)
    
    if sample_path.exists():
        # 샘플 데이터 정보 확인
        filelist_path = sample_path / "FileList.csv"
        video_dir = sample_path / "Videos"
        
        if filelist_path.exists() and video_dir.exists():
            import pandas as pd
            df = pd.read_csv(filelist_path)
            video_count = len(list(video_dir.glob("*.avi")))
            
            print(f"✅ Sample dataset found:")
            print(f"  Location: {DATA_ROOT}")
            print(f"  Videos in FileList.csv: {len(df)}")
            print(f"  Video files in Videos/: {video_count}")
            
            # Split distribution 확인
            if 'Split' in df.columns:
                print(f"\nSplit distribution:")
                for split in ['TRAIN', 'VAL', 'TEST']:
                    count = len(df[df['Split'] == split])
                    print(f"  {split}: {count}")
            else:
                print(f"\nFileList columns: {df.columns.tolist()}")
        else:
            print(f"❌ Sample dataset structure incorrect!")
            print(f"  Expected: {filelist_path}")
            print(f"  Expected: {video_dir}")
            DATA_ROOT = None
    else:
        print(f"❌ Sample dataset not found at {DATA_ROOT}")
        print("Please check the path or upload sample data to Drive")
        DATA_ROOT = None
else:
    # 전체 데이터 사용
    DATA_ROOT = "/content/drive/MyDrive/echonet_dynamic"  # 여기를 실제 경로로 수정


## 3. 설정 및 모듈 임포트


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from pathlib import Path
import json
from datetime import datetime
import sys

# 프로젝트 경로 추가
sys.path.append('/content/2025_edge_computing_task2')

# 모듈 임포트
import config
from dataset import get_dataloaders
from model import create_model
from train import train_model, load_checkpoint
from quant_utils import apply_ptq, apply_qat
from metrics import evaluate_model_performance
from main_ptq import run_ptq_experiment
from main_qat import run_qat_experiment
from run_all import run_all_experiments

# Colab용 설정 업데이트
# 위에서 설정한 DATA_ROOT를 사용
if 'DATA_ROOT' in locals() and DATA_ROOT is not None:
    config.BASE_DIR = Path(DATA_ROOT)
    config.VIDEO_DIR = config.BASE_DIR / "Videos"
    config.FILELIST_PATH = config.BASE_DIR / "FileList.csv"
    print(f"✅ Using data from: {config.BASE_DIR}")
else:
    print("❌ DATA_ROOT not set! Please run data preparation cells above.")
    raise ValueError("DATA_ROOT must be set. Check data preparation section.")

# 디렉토리 생성
config.RESULTS_DIR.mkdir(exist_ok=True, parents=True)
config.CHECKPOINT_DIR.mkdir(exist_ok=True, parents=True)

# 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.DEVICE = device

print(f"Using device: {device}")
print(f"Data root: {config.BASE_DIR}")
print(f"Video dir: {config.VIDEO_DIR}")
print(f"FileList: {config.FILELIST_PATH}")


## 4. 데이터 로딩


In [ ]:
# 데이터 경로 확인
if not config.VIDEO_DIR.exists():
    raise FileNotFoundError(f"Video directory not found: {config.VIDEO_DIR}")
if not config.FILELIST_PATH.exists():
    raise FileNotFoundError(f"FileList.csv not found: {config.FILELIST_PATH}")

# 데이터 로더 생성
print("Loading dataset...")
train_loader, val_loader = get_dataloaders(
    video_dir=config.VIDEO_DIR,
    filelist_path=config.FILELIST_PATH,
    batch_size=config.BATCH_SIZE,
    num_workers=0,  # Colab에서는 0 권장
    num_frames=config.NUM_FRAMES,
    img_size=config.IMG_SIZE,
)

print(f"Train samples: {len(train_loader.dataset)}")
print(f"Val samples: {len(val_loader.dataset)}")

# Latency 측정용 샘플 입력 준비
sample_videos, _ = next(iter(val_loader))
sample_input = sample_videos[:1].to(device)
print(f"\nSample video shape: {sample_videos.shape}")  # (B, N, C, H, W)


## 5. Baseline 모델 학습


In [ ]:
# Baseline 모델 생성
baseline_model = create_model()
print(f"Model created: {baseline_model.__class__.__name__}")

# 모델 파라미터 수 확인
total_params = sum(p.numel() for p in baseline_model.parameters())
trainable_params = sum(p.numel() for p in baseline_model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

baseline_checkpoint_path = config.CHECKPOINT_DIR / "best_model.pth"


In [ ]:
# Baseline 모델 학습 또는 로드
if not baseline_checkpoint_path.exists():
    print("Training baseline model...")
    history = train_model(
        baseline_model,
        train_loader,
        val_loader,
        num_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        weight_decay=config.WEIGHT_DECAY,
        device=device,
        checkpoint_dir=config.CHECKPOINT_DIR,
        save_best=True,
        verbose=True,
    )
    
    # 최종 모델 저장
    torch.save(baseline_model.state_dict(), baseline_checkpoint_path)
    print(f"Baseline model saved to {baseline_checkpoint_path}")
else:
    print(f"Loading baseline model from {baseline_checkpoint_path}")
    baseline_model.load_state_dict(torch.load(baseline_checkpoint_path, map_location=device))
    baseline_model = baseline_model.to(device)


In [ ]:
# 학습 히스토리 시각화 (optional)
import matplotlib.pyplot as plt

if 'history' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss
    axes[0].plot(history['train_loss'], label='Train Loss')
    axes[0].plot(history['val_loss'], label='Val Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # MAE
    axes[1].plot(history['val_mae'], label='Val MAE')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('MAE')
    axes[1].set_title('Validation MAE')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()


## 6. Baseline 모델 평가


In [ ]:
# Baseline 평가
print("Evaluating baseline model...")
baseline_model.eval()
criterion = nn.MSELoss()
baseline_performance = evaluate_model_performance(
    baseline_model,
    val_loader,
    criterion,
    device=device,
    verbose=True,
)

baseline_summary = {
    'model_type': 'baseline',
    'MAE': baseline_performance['mae'],
    'size_mb': baseline_performance['size_mb'],
    'latency_ms_per_video': baseline_performance['latency_ms']
}

print(f"\nBaseline Results:")
print(f"  MAE: {baseline_summary['MAE']:.4f}")
print(f"  Model Size: {baseline_summary['size_mb']:.4f} MB")
print(f"  Latency: {baseline_summary['latency_ms_per_video']:.4f} ms/video")


## 7. Post-Training Quantization (PTQ)


In [ ]:
# PTQ 실험 실행
ptq_results = run_ptq_experiment(
    baseline_checkpoint_path=baseline_checkpoint_path,
    save_results=True,
    verbose=True,
)


## 8. Quantization-Aware Training (QAT)


In [ ]:
# QAT 실험 실행
qat_results = run_qat_experiment(
    baseline_checkpoint_path=baseline_checkpoint_path,
    save_results=True,
    verbose=True,
)


## 9. 최종 비교 결과


In [ ]:
# 전체 결과 비교 테이블 생성
import pandas as pd

comparison_data = [
    {
        'Model': 'FP32 Baseline',
        'Precision': 'FP32',
        'Size (MB)': f"{baseline_summary['size_mb']:.4f}",
        'MAE': f"{baseline_summary['MAE']:.4f}",
        'Latency (ms)': f"{baseline_summary['latency_ms_per_video']:.4f}",
    },
    {
        'Model': 'PTQ',
        'Precision': 'INT8',
        'Size (MB)': f"{ptq_results['ptq']['size_mb']:.4f}",
        'MAE': f"{ptq_results['ptq']['mae']:.4f}",
        'Latency (ms)': f"{ptq_results['ptq']['latency_ms']:.4f}",
    },
    {
        'Model': 'QAT',
        'Precision': 'INT8',
        'Size (MB)': f"{qat_results['qat']['size_mb']:.4f}",
        'MAE': f"{qat_results['qat']['mae']:.4f}",
        'Latency (ms)': f"{qat_results['qat']['latency_ms']:.4f}",
    },
]

comparison_df = pd.DataFrame(comparison_data)
print("="*60)
print("Final Comparison Results")
print("="*60)
print(comparison_df.to_string(index=False))


In [ ]:
# 상세 비교 분석
print("\n" + "="*60)
print("Detailed Comparison Analysis")
print("="*60)

print(f"\n1. Accuracy (MAE):")
print(f"   Baseline: {baseline_summary['MAE']:.4f}")
print(f"   PTQ:      {ptq_results['ptq']['mae']:.4f} (Drop: {ptq_results['ptq']['mae_drop']:.4f}, {ptq_results['ptq']['mae_drop_percent']:.2f}%)")
print(f"   QAT:      {qat_results['qat']['mae']:.4f} (Drop: {qat_results['qat']['mae_drop']:.4f}, {qat_results['qat']['mae_drop_percent']:.2f}%)")

mae_diff = ptq_results['ptq']['mae'] - qat_results['qat']['mae']
print(f"   QAT vs PTQ: {mae_diff:.4f} {'(QAT better)' if mae_diff > 0 else '(PTQ better)'}")

print(f"\n2. Model Size:")
print(f"   Baseline: {baseline_summary['size_mb']:.4f} MB")
print(f"   PTQ:      {ptq_results['ptq']['size_mb']:.4f} MB (Reduction: {ptq_results['ptq']['size_reduction_mb']:.4f} MB, {ptq_results['ptq']['size_reduction_percent']:.2f}%)")
print(f"   QAT:      {qat_results['qat']['size_mb']:.4f} MB (Reduction: {qat_results['qat']['size_reduction_mb']:.4f} MB, {qat_results['qat']['size_reduction_percent']:.2f}%)")

print(f"\n3. Latency:")
print(f"   Baseline: {baseline_summary['latency_ms_per_video']:.4f} ms/video")
print(f"   PTQ:      {ptq_results['ptq']['latency_ms']:.4f} ms/video (Improvement: {ptq_results['ptq']['latency_improvement_ms']:.4f} ms, {ptq_results['ptq']['latency_improvement_percent']:.2f}%)")
print(f"   QAT:      {qat_results['qat']['latency_ms']:.4f} ms/video (Improvement: {qat_results['qat']['latency_improvement_ms']:.4f} ms, {qat_results['qat']['latency_improvement_percent']:.2f}%)")


## 10. 결과 시각화 (Optional)


In [ ]:
# 결과 시각화
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

models = ['Baseline\n(FP32)', 'PTQ\n(INT8)', 'QAT\n(INT8)']
x = np.arange(len(models))

# MAE 비교
mae_values = [
    baseline_summary['MAE'],
    ptq_results['ptq']['mae'],
    qat_results['qat']['mae'],
]
axes[0].bar(x, mae_values, color=['blue', 'orange', 'green'])
axes[0].set_ylabel('MAE')
axes[0].set_title('Mean Absolute Error')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models)
axes[0].grid(True, axis='y', alpha=0.3)

# Model Size 비교
size_values = [
    baseline_summary['size_mb'],
    ptq_results['ptq']['size_mb'],
    qat_results['qat']['size_mb'],
]
axes[1].bar(x, size_values, color=['blue', 'orange', 'green'])
axes[1].set_ylabel('Size (MB)')
axes[1].set_title('Model Size')
axes[1].set_xticks(x)
axes[1].set_xticklabels(models)
axes[1].grid(True, axis='y', alpha=0.3)

# Latency 비교
latency_values = [
    baseline_summary['latency_ms_per_video'],
    ptq_results['ptq']['latency_ms'],
    qat_results['qat']['latency_ms'],
]
axes[2].bar(x, latency_values, color=['blue', 'orange', 'green'])
axes[2].set_ylabel('Latency (ms/video)')
axes[2].set_title('Inference Latency')
axes[2].set_xticks(x)
axes[2].set_xticklabels(models)
axes[2].grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


## 11. 결과 파일 다운로드 (Colab)


In [ ]:
# 결과 파일 다운로드
from google.colab import files
import zipfile
from pathlib import Path

# 결과 디렉토리 확인
results_dir = config.RESULTS_DIR
print(f"Results directory: {results_dir}")
print(f"Files in results directory:")
for f in results_dir.glob("*"):
    print(f"  - {f.name}")

# 결과 파일들을 zip으로 압축
zip_path = "/content/quantization_results.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file_path in results_dir.glob("*"):
        zipf.write(file_path, file_path.name)
    # 체크포인트도 포함 (선택사항)
    # for file_path in config.CHECKPOINT_DIR.glob("*.pth"):
    #     zipf.write(file_path, f"checkpoints/{file_path.name}")

print(f"\nResults zipped to: {zip_path}")

# 다운로드
files.download(zip_path)


## 12. 전체 실험 한 번에 실행 (Alternative)

위의 단계별 실행 대신, `run_all.py`를 사용하여 전체 실험을 한 번에 실행할 수도 있습니다.


In [ ]:
# 전체 실험 한 번에 실행 (선택사항)
# 주의: 이 방법은 baseline을 새로 학습합니다
# 이미 학습된 모델이 있다면 위의 단계별 실행을 권장합니다

# all_results = run_all_experiments(
#     train_baseline=True,  # False로 설정하면 기존 체크포인트 사용
#     save_results=True,
#     verbose=True,
# )
